# 🔹 UFC Fight Predictor Model Training

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 1. Import Libraries and Setup Environment

In [1]:
# Import necessary libraries
import os
import sys
import pandas as pd

pd.set_option('display.max_colwidth', 200) 

# Get the current working directory
current_dir = os.getcwd()

# Navigate to the project root
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Import from /src
sys.path.append(os.path.join(project_root))
from src.metrics import *
from src.model_factory import *
from src.io_model import *
from src.helpers import *
from src.config import *

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 2. Load Data

In [2]:
# Load UFCData
try:
    UFCData = load_data()
except Exception as e:
    print_header(f"Error loading training data: {e}", color='bright_red')

/home/mlioi/ufc-predictor/data/processed/ufc_data.pkl
📦 UFCData object loaded from: /home/mlioi/ufc-predictor/data/processed/ufc_data.pkl


In [3]:
UFCData

📊 UFC Dataset Summary
----------------------------------------
🧪 Total samples      : 6001
🧪 Train/Test split  : 4800 / 1201
🧪 Total features     : 20

🔢 Numerical features : 17
🔠 Categorical features: 3
    - Binary          : 2
    - Multiclass      : 1

🏷 Label distribution (raw):
   - Class 0: 3484 (58.1%)
   - Class 1: 2517 (41.9%)

✅ No missing values detected

📈 Feature summary statistics (train set):
                        mean      std       min       max
LoseStreakDif          0.059    1.012    -6.000     6.000
WinStreakDif          -0.162    1.921   -18.000    10.000
TotalTitleBoutDif     -0.313    1.709   -16.000    15.000
KODif                 -0.536    2.181   -21.000    14.000
SubDif                -0.334    1.877   -15.000    10.000
HeightDif             -0.030    6.286   -33.020    30.480
ReachDif              -0.260    8.266   -33.020    30.480
AgeDif                 0.196    5.213   -17.000    17.000
SigStrDif             -2.540   19.869  -118.000   128.222
AvgSubAt

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 3. 🔧 Hyperparameters Tuning 🔧

In [4]:
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

hyperparameter_tuning = True

if hyperparameter_tuning:
    print_header('Hyperparameters Tuning is Activated', color = 'bright_blue')
    model_params = {
    "Support Vector Machine": (
        SVC(probability=True),
        {'C': [0.01, 0.1, 1, 1.5, 2], 'kernel': ['linear'], 'gamma': ['auto']}
    ),
    "Random Forest": (
        RandomForestClassifier(),
        {'n_estimators': [10, 50, 100, 1000], 'max_depth': [3, 5, 10, 100]}
    ),
    "Logistic Regression": (
        LogisticRegression(),
        {'C': [0.001, 0.01, 0.05, 1], 'solver': ['liblinear'], 'penalty': ['l1','l2']}
    ),
    "K-Nearest Neighbors": (
        KNeighborsClassifier(),
        {'n_neighbors': [3, 6, 9, 12, 15], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']}
    ),
    "AdaBoost": (
        AdaBoostClassifier(),
        {'n_estimators': [10, 50, 100], 'learning_rate': [0.01, 0.1, 1.0]}
    ),
    "Naive Bayes": (
        GaussianNB(),
        {'var_smoothing': [1e-8, 1e-7, 1e-6, 1e-5]}
    ),
    "Gradient Boosting": (
        GradientBoostingClassifier(),
        {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1, 1.0], 'max_depth': [3, 5, 7]}
    ),
    "Extra Trees": (
        ExtraTreesClassifier(),
        {'n_estimators': [50, 100], 'max_depth': [None, 10, 20]}
    ),
    "Quadratic Discriminant Analysis": (
        QuadraticDiscriminantAnalysis(),
        {'reg_param': [0.1, 1]}
    ),
    "Neural Network": (
        MLPClassifier(max_iter=200, random_state=42),
        {
        # Architecture: number and size of hidden layers
        'hidden_layer_sizes': [
            (200,), (50, 50),
        ],
        # Activation function for hidden layers
        'activation': ['relu', 'logistic'],
        # Optimizer for gradient descent
        'solver': ['adam', 'sgd'],
        # L2 regularization strength (higher alpha reduces overfitting but can cause underfitting)
        'alpha': [0.001, 0.01],
        # Learning rate schedule
        'learning_rate': ['adaptive'],
        # Initial learning rate (for adam and sgd solvers)
        'learning_rate_init': [0.01],
        # Early stopping based on validation performance to prevent overfitting
        'early_stopping': [True],
        # Mini-batch size for training with adam/sgd
        'batch_size': [32, 64],
        # Momentum for SGD (controls contribution of previous updates)
        'momentum': [0.8],
        # Proportion of training set used as validation for early stopping
        'validation_fraction': [0.2]
        }
    ),
    "XGBoost": (
            XGBClassifier(eval_metric='logloss'),
            {
                'n_estimators': [50, 150],
                'learning_rate': [0.01, 0.1],
                'max_depth': [3, 5, 7],
                'subsample': [0.8, 1.0],
                'colsample_bytree': [0.8, 1.0]
            }
        )
}
    display_model_params_table(model_params)
else:
    print_header('Hyperparameters Tuning Off', color = 'bright_red')
    model_params = None # Default Settings

╔═══════════════════════════════════════╗
║  Hyperparameters Tuning is Activated  ║
╚═══════════════════════════════════════╝


,Model,Hyperparameters
0,Support Vector Machine,"C: [0.01, 0.1, 1, 1.5, 2]; kernel: ['linear']; gamma: ['auto']"
1,Random Forest,"n_estimators: [10, 50, 100, 1000]; max_depth: [3, 5, 10, 100]"
2,Logistic Regression,"C: [0.001, 0.01, 0.05, 1]; solver: ['liblinear']; penalty: ['l1', 'l2']"
3,K-Nearest Neighbors,"n_neighbors: [3, 6, 9, 12, 15]; weights: ['uniform', 'distance']; metric: ['euclidean', 'manhattan']"
4,AdaBoost,"n_estimators: [10, 50, 100]; learning_rate: [0.01, 0.1, 1.0]"
5,Naive Bayes,"var_smoothing: [1e-08, 1e-07, 1e-06, 1e-05]"
6,Gradient Boosting,"n_estimators: [50, 100]; learning_rate: [0.01, 0.1, 1.0]; max_depth: [3, 5, 7]"
7,Extra Trees,"n_estimators: [50, 100]; max_depth: [None, 10, 20]"
8,Quadratic Discriminant Analysis,"reg_param: [0.1, 1]"
9,Neural Network,"hidden_layer_sizes: [(200,), (50, 50)]; activation: ['relu', 'logistic']; solver: ['adam', 'sgd']; alpha: [0.001, 0.01]; learning_rate: ['adaptive']; learning_rate_init: [0.01]; early_stopping: [T..."


## Notes
- For SVM, kernel='linear' has better performance.
- For Logistic Regression, performs well for lower values of c

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 4. 🔹 UFC Machine Learning Training

### 4.1 KNN Model Training

#### 🚀 KNN Training 

In [5]:
model_name = 'K-Nearest Neighbors'
start = time.time()
model = model_factory(model_name, UFCData, model_params, scoring = 'accuracy')
end = time.time()
duration = end - start

╔══════════════════════════════════════════════════════════════╗
║  [K-Nearest Neighbors] UFC GridSearchCV Training (Accuracy)  ║
╚══════════════════════════════════════════════════════════════╝


INFO:root:[K-Nearest Neighbors] 🤖 Training...


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.570 total time=   0.1s
[CV 2/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.591 total time=   0.1s
[CV 3/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.621 total time=   0.1s
[CV 4/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.597 total time=   0.1s
[CV 5/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.599 total time=   0.1s
[CV 1/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.570 total time=   0.0s
[CV 2/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.593 total time=   0.0s
[CV 3/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.621 total time=   0.0s
[CV 4/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.597 total time=   0.0s
[CV 5/5] END metric=euclidean, n_neighbors=3, weights=distance;, score=0.598 to

INFO:root:[K-Nearest Neighbors] 🔍 Best Accuracy: 0.6250
[K-Nearest Neighbors] 🔍 Best Params: {'metric': 'euclidean', 'n_neighbors': 15, 'weights': 'distance'}


#### 🔍 KNN Metrics

In [6]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [7]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [8]:
save_model(model, name=file_model_name[model_name])

✅ Model K-Nearest Neighbors saved to: /home/mlioi/ufc-predictor/models/knn_best.pkl


### 4.2 Support Vector Machine Model 

#### 🚀 Support Vector Machine Training 

In [9]:
model_name = 'Support Vector Machine'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔═════════════════════════════════════════════════════════════════╗
║  [Support Vector Machine] UFC GridSearchCV Training (Accuracy)  ║
╚═════════════════════════════════════════════════════════════════╝


INFO:root:[Support Vector Machine] 🤖 Training...


Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.652 total time=   2.8s
[CV 2/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.643 total time=   2.8s
[CV 3/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.692 total time=   2.8s
[CV 4/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.665 total time=   2.7s
[CV 5/5] END .C=0.01, gamma=auto, kernel=linear;, score=0.659 total time=   2.7s
[CV 1/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.654 total time=   3.0s
[CV 2/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.637 total time=   3.0s
[CV 3/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.688 total time=   3.1s
[CV 4/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.661 total time=   3.0s
[CV 5/5] END ..C=0.1, gamma=auto, kernel=linear;, score=0.659 total time=   3.1s
[CV 1/5] END ....C=1, gamma=auto, kernel=linear;, score=0.656 total time=   5.2s
[CV 2/5] END ....C=1, gamma=auto, kernel=linear;,

INFO:root:[Support Vector Machine] 🔍 Best Accuracy: 0.6621
[Support Vector Machine] 🔍 Best Params: {'C': 0.01, 'gamma': 'auto', 'kernel': 'linear'}


#### 🔍 Support Vector Machine Metrics

In [10]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [11]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [12]:
save_model(model, name=file_model_name[model_name])

✅ Model Support Vector Machine saved to: /home/mlioi/ufc-predictor/models/svm_best.pkl


### 4.3 Logistic Regression Model

#### 🚀 Logistic Regression Training

In [13]:
model_name = 'Logistic Regression'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔══════════════════════════════════════════════════════════════╗
║  [Logistic Regression] UFC GridSearchCV Training (Accuracy)  ║
╚══════════════════════════════════════════════════════════════╝


INFO:root:[Logistic Regression] 🤖 Training...


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.581 total time=   0.0s
[CV 2/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.581 total time=   0.0s
[CV 3/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.580 total time=   0.0s
[CV 4/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.580 total time=   0.0s
[CV 5/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.580 total time=   0.0s
[CV 1/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.656 total time=   0.0s
[CV 2/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.636 total time=   0.0s
[CV 3/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.682 total time=   0.0s
[CV 4/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.657 total time=   0.0s
[CV 5/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.669 total time=   0.0s
[CV 1/5] END C=0.01, penalty=l1, solver=liblinear;, score=0.657 total time=   0.0s
[CV 2/5] END C=0.

INFO:root:[Logistic Regression] 🔍 Best Accuracy: 0.6638
[Logistic Regression] 🔍 Best Params: {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}


#### 🔍 Logistic Regression Metrics

In [14]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [15]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [16]:
save_model(model, name=file_model_name[model_name])

✅ Model Logistic Regression saved to: /home/mlioi/ufc-predictor/models/lr_best.pkl


### 4.4 Random Forest Model

#### 🚀 Random Forest Training

In [17]:
model_name = 'Random Forest'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔════════════════════════════════════════════════════════╗
║  [Random Forest] UFC GridSearchCV Training (Accuracy)  ║
╚════════════════════════════════════════════════════════╝


INFO:root:[Random Forest] 🤖 Training...


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ......max_depth=3, n_estimators=10;, score=0.655 total time=   0.0s
[CV 2/5] END ......max_depth=3, n_estimators=10;, score=0.585 total time=   0.0s
[CV 3/5] END ......max_depth=3, n_estimators=10;, score=0.649 total time=   0.0s
[CV 4/5] END ......max_depth=3, n_estimators=10;, score=0.650 total time=   0.0s
[CV 5/5] END ......max_depth=3, n_estimators=10;, score=0.660 total time=   0.0s
[CV 1/5] END ......max_depth=3, n_estimators=50;, score=0.644 total time=   0.1s
[CV 2/5] END ......max_depth=3, n_estimators=50;, score=0.625 total time=   0.1s
[CV 3/5] END ......max_depth=3, n_estimators=50;, score=0.660 total time=   0.1s
[CV 4/5] END ......max_depth=3, n_estimators=50;, score=0.646 total time=   0.1s
[CV 5/5] END ......max_depth=3, n_estimators=50;, score=0.623 total time=   0.1s
[CV 1/5] END .....max_depth=3, n_estimators=100;, score=0.639 total time=   0.3s
[CV 2/5] END .....max_depth=3, n_estimators=100;

INFO:root:[Random Forest] 🔍 Best Accuracy: 0.6606
[Random Forest] 🔍 Best Params: {'max_depth': 10, 'n_estimators': 100}


#### 🔍 Random Forest Metrics

In [18]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [19]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [20]:
save_model(model, name=file_model_name[model_name])

✅ Model Random Forest saved to: /home/mlioi/ufc-predictor/models/rf_best.pkl


### 4.5 AdaBoost Model

#### 🚀 AdaBoost Training

In [21]:
model_name = 'AdaBoost'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔═══════════════════════════════════════════════════╗
║  [AdaBoost] UFC GridSearchCV Training (Accuracy)  ║
╚═══════════════════════════════════════════════════╝


INFO:root:[AdaBoost] 🤖 Training...


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END learning_rate=0.01, n_estimators=10;, score=0.650 total time=   0.1s
[CV 2/5] END learning_rate=0.01, n_estimators=10;, score=0.640 total time=   0.1s
[CV 3/5] END learning_rate=0.01, n_estimators=10;, score=0.648 total time=   0.1s
[CV 4/5] END learning_rate=0.01, n_estimators=10;, score=0.647 total time=   0.1s
[CV 5/5] END learning_rate=0.01, n_estimators=10;, score=0.610 total time=   0.1s
[CV 1/5] END learning_rate=0.01, n_estimators=50;, score=0.650 total time=   0.3s
[CV 2/5] END learning_rate=0.01, n_estimators=50;, score=0.640 total time=   0.3s
[CV 3/5] END learning_rate=0.01, n_estimators=50;, score=0.648 total time=   0.3s
[CV 4/5] END learning_rate=0.01, n_estimators=50;, score=0.647 total time=   0.3s
[CV 5/5] END learning_rate=0.01, n_estimators=50;, score=0.610 total time=   0.3s
[CV 1/5] END learning_rate=0.01, n_estimators=100;, score=0.650 total time=   0.6s
[CV 2/5] END learning_rate=0.01, n_es

INFO:root:[AdaBoost] 🔍 Best Accuracy: 0.6663
[AdaBoost] 🔍 Best Params: {'learning_rate': 0.1, 'n_estimators': 100}


#### 🔍 AdaBoost Metrics

In [22]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [23]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [24]:
save_model(model, name=file_model_name[model_name])

✅ Model AdaBoost saved to: /home/mlioi/ufc-predictor/models/ab_best.pkl


### 4.6 Naive Bayes Model

#### 🚀 Naive Bayes Training

In [25]:
model_name = 'Naive Bayes'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔══════════════════════════════════════════════════════╗
║  [Naive Bayes] UFC GridSearchCV Training (Accuracy)  ║
╚══════════════════════════════════════════════════════╝


INFO:root:[Naive Bayes] 🤖 Training...


Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ...............var_smoothing=1e-08;, score=0.643 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-08;, score=0.629 total time=   0.0s
[CV 3/5] END ...............var_smoothing=1e-08;, score=0.630 total time=   0.0s
[CV 4/5] END ...............var_smoothing=1e-08;, score=0.621 total time=   0.0s
[CV 5/5] END ...............var_smoothing=1e-08;, score=0.642 total time=   0.0s
[CV 1/5] END ...............var_smoothing=1e-07;, score=0.643 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-07;, score=0.629 total time=   0.0s
[CV 3/5] END ...............var_smoothing=1e-07;, score=0.630 total time=   0.0s
[CV 4/5] END ...............var_smoothing=1e-07;, score=0.621 total time=   0.0s
[CV 5/5] END ...............var_smoothing=1e-07;, score=0.642 total time=   0.0s
[CV 1/5] END ...............var_smoothing=1e-06;, score=0.643 total time=   0.0s
[CV 2/5] END ...............var_smoothing=1e-06;,

INFO:root:[Naive Bayes] 🔍 Best Accuracy: 0.6329
[Naive Bayes] 🔍 Best Params: {'var_smoothing': 1e-08}


#### 🔍 Naive Bayes Metrics

In [26]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [27]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [28]:
save_model(model, name=file_model_name[model_name])

✅ Model Naive Bayes saved to: /home/mlioi/ufc-predictor/models/nb_best.pkl


### 4.7 Gradient Boosting Model

#### 🚀 Gradient Boosting Training

In [29]:
model_name = "Gradient Boosting"
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔════════════════════════════════════════════════════════════╗
║  [Gradient Boosting] UFC GridSearchCV Training (Accuracy)  ║
╚════════════════════════════════════════════════════════════╝


INFO:root:[Gradient Boosting] 🤖 Training...


Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.637 total time=   0.6s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.642 total time=   0.6s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.681 total time=   0.6s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.650 total time=   0.6s
[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.644 total time=   0.6s
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.642 total time=   1.1s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.639 total time=   1.1s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.699 total time=   1.1s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.666 total time=   1.1s
[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.652 tot

INFO:root:[Gradient Boosting] 🔍 Best Accuracy: 0.6594
[Gradient Boosting] 🔍 Best Params: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}


#### 🔍 Gradient Boosting Metrics

In [30]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [31]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [32]:
save_model(model, name=file_model_name[model_name])

✅ Model Gradient Boosting saved to: /home/mlioi/ufc-predictor/models/gb_best.pkl


### 4.8 Extra Trees Model

#### 🚀 Extra Trees Training

In [33]:
model_name = "Extra Trees"
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔══════════════════════════════════════════════════════╗
║  [Extra Trees] UFC GridSearchCV Training (Accuracy)  ║
╚══════════════════════════════════════════════════════╝


INFO:root:[Extra Trees] 🤖 Training...


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ...max_depth=None, n_estimators=50;, score=0.647 total time=   0.3s
[CV 2/5] END ...max_depth=None, n_estimators=50;, score=0.632 total time=   0.3s
[CV 3/5] END ...max_depth=None, n_estimators=50;, score=0.648 total time=   0.3s
[CV 4/5] END ...max_depth=None, n_estimators=50;, score=0.633 total time=   0.3s
[CV 5/5] END ...max_depth=None, n_estimators=50;, score=0.622 total time=   0.3s
[CV 1/5] END ..max_depth=None, n_estimators=100;, score=0.640 total time=   0.5s
[CV 2/5] END ..max_depth=None, n_estimators=100;, score=0.630 total time=   0.5s
[CV 3/5] END ..max_depth=None, n_estimators=100;, score=0.660 total time=   0.5s
[CV 4/5] END ..max_depth=None, n_estimators=100;, score=0.640 total time=   0.6s
[CV 5/5] END ..max_depth=None, n_estimators=100;, score=0.627 total time=   0.5s
[CV 1/5] END .....max_depth=10, n_estimators=50;, score=0.641 total time=   0.1s
[CV 2/5] END .....max_depth=10, n_estimators=50;,

INFO:root:[Extra Trees] 🔍 Best Accuracy: 0.6483
[Extra Trees] 🔍 Best Params: {'max_depth': 20, 'n_estimators': 100}


#### 🔍 Extra Trees Metrics

In [34]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [35]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [36]:
save_model(model, name=file_model_name[model_name])

✅ Model Extra Trees saved to: /home/mlioi/ufc-predictor/models/et_best.pkl


### 4.9 Quadratic Discriminant Analysis Model

#### 🚀 Quadratic Discriminant Analysis Training

In [37]:
model_name = "Quadratic Discriminant Analysis"
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔══════════════════════════════════════════════════════════════════════════╗
║  [Quadratic Discriminant Analysis] UFC GridSearchCV Training (Accuracy)  ║
╚══════════════════════════════════════════════════════════════════════════╝


INFO:root:[Quadratic Discriminant Analysis] 🤖 Training...


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END .....................reg_param=0.1;, score=0.632 total time=   0.0s
[CV 2/5] END .....................reg_param=0.1;, score=0.637 total time=   0.0s
[CV 3/5] END .....................reg_param=0.1;, score=0.665 total time=   0.0s
[CV 4/5] END .....................reg_param=0.1;, score=0.624 total time=   0.0s
[CV 5/5] END .....................reg_param=0.1;, score=0.641 total time=   0.0s
[CV 1/5] END .......................reg_param=1;, score=0.654 total time=   0.0s
[CV 2/5] END .......................reg_param=1;, score=0.626 total time=   0.0s
[CV 3/5] END .......................reg_param=1;, score=0.648 total time=   0.0s
[CV 4/5] END .......................reg_param=1;, score=0.642 total time=   0.0s
[CV 5/5] END .......................reg_param=1;, score=0.658 total time=   0.0s


INFO:root:[Quadratic Discriminant Analysis] 🔍 Best Accuracy: 0.6456
[Quadratic Discriminant Analysis] 🔍 Best Params: {'reg_param': 1}


#### 🔍 Quadratic Discriminant Analysis Metrics

In [38]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [39]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [40]:
save_model(model, name=file_model_name[model_name])

✅ Model Quadratic Discriminant Analysis saved to: /home/mlioi/ufc-predictor/models/qda_best.pkl


### 4.10 Neural Network

#### 🚀 Neural Network Training 

In [41]:
model_name = 'Neural Network'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔═════════════════════════════════════════════════════════╗
║  [Neural Network] UFC GridSearchCV Training (Accuracy)  ║
╚═════════════════════════════════════════════════════════╝


INFO:root:[Neural Network] 🤖 Training...


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.2;, score=0.654 total time=   0.8s
[CV 2/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.2;, score=0.644 total time=   0.6s
[CV 3/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.2;, score=0.662 total time=   0.6s
[CV 4/5] END activation=relu, alpha=0.001, batch_size=32, early_stopping=True, hidden_layer_sizes=(200,), learning_rate=adaptive, learning_rate_init=0.01, momentum=0.8, solver=adam, validation_fraction=0.2;, score=0.645 t

INFO:root:[Neural Network] 🔍 Best Accuracy: 0.6621
[Neural Network] 🔍 Best Params: {'activation': 'logistic', 'alpha': 0.001, 'batch_size': 32, 'early_stopping': True, 'hidden_layer_sizes': (50, 50), 'learning_rate': 'adaptive', 'learning_rate_init': 0.01, 'momentum': 0.8, 'solver': 'adam', 'validation_fraction': 0.2}


#### 🔍 Neural Network Metrics

In [42]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [43]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [44]:
save_model(model, name=file_model_name[model_name])

✅ Model Neural Network saved to: /home/mlioi/ufc-predictor/models/nn_best.pkl


### 4.11 XGBoost Model

#### 🚀 XGBoost Training 

In [45]:
model_name = 'XGBoost'
start = time.time()
model = model_factory(model_name, UFCData, model_params)
end = time.time()
duration = end - start

╔══════════════════════════════════════════════════╗
║  [XGBoost] UFC GridSearchCV Training (Accuracy)  ║
╚══════════════════════════════════════════════════╝


INFO:root:[XGBoost] 🤖 Training...


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.626 total time=   0.3s
[CV 2/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.620 total time=   0.1s
[CV 3/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.617 total time=   0.1s
[CV 4/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.611 total time=   0.1s
[CV 5/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8;, score=0.636 total time=   0.1s
[CV 1/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0;, score=0.624 total time=   0.1s
[CV 2/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0;, score=0.628 total time=   0.1s
[CV 3/5] END c

INFO:root:[XGBoost] 🔍 Best Accuracy: 0.6635
[XGBoost] 🔍 Best Params: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}


#### 🔍 XGBoost Metrics

In [46]:
metrics = evaluate_metrics(model, UFCData)

### 📝 Log Training Results

In [47]:
log_training_result(model_name, model.best_params_, metrics, duration)

✅ Training logged to ../data/results/training_log.csv


### 💾 Save Model 

In [48]:
save_model(model, name=file_model_name[model_name])

✅ Model XGBoost saved to: /home/mlioi/ufc-predictor/models/xgb_best.pkl


<div style="text-align: center;">
     <img src="../img/ufc_logo.png" width="800" /> 
</div>